In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!nvidia-smi

Mon Aug  8 08:08:40 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 14.4 MB/s 
     |████████████████████████████████| 6.6 MB 47.9 MB/s 
     |████████████████████████████████| 596 kB 18.2 MB/s 
     |████████████████████████████████| 101 kB 11.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import shutil
import sys
import re

# 1. Pre Processing input data

In [ ]:
# replace 100k or 100d to 'giá tiền'
def find_money(x):
  count_num = 0
  for char in x:
    if count_num ==0:
      if char.isalpha():
        return x
      else:
        count_num +=1
    elif char.isalpha() and  char in ['K', 'k', 'd', 'đ', 'D', 'Đ']:
      return ''.join('giá tiền')
    elif char.isnumeric() or char == '.':
      count_num =1
  return x
  
def replace_money(txt):
  list_txt = txt.split(' ')
  tmp = []
  for word in list_txt:
    tmp.append(find_money(word))
  return ' '.join(tmp)

# clean emoijs
def remove_emojis(txt):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', txt)

def change_emoijs(x):
  emoij_pos = ['😹', '😍', '💘', '👍', '😜', '❤️', '🌸', '😊', '😚', '😘', '😝', '😄', '🎉', '👍🏻', '😑', '😻', '😙', '😁', '😌', '😱', '😜', '😋', '💖', '💕', '💋','👌🏼', '😏', '😎', '😄']
  emoij_neg = ['🙄','😢','😑','👎','😭','😪','😞', '😅']
  for emoij_p in emoij_pos:
    x = re.sub(emoij_p, ' thích ', x)
  for emoij_n in emoij_neg:
    x = re.sub(emoij_n, ' không thích ', x)
  return remove_emojis(x)

# special characters
def remove_special_char(txt):
  txt = txt.replace('=', ' ')
  txt = txt.replace('?', ' ')
  txt = txt.replace('#', ' ')
  txt = txt.replace('+', ' ')
  txt = txt.replace('-', ' ')
  txt = txt.replace('!', '')
  txt = txt.replace('$', ' ')
  txt = txt.replace('@', ' ')
  txt = txt.replace('&', ' ')
  txt = txt.replace('>', ' ')
  txt = txt.replace('<', ' ')
  txt = txt.replace(')', '')
  txt = txt.replace('(', ' ')
  txt = txt.replace(u'•', ' ')
  return txt
# 'k', 'khong', 'ko', 'khong', 'kg' -> 'Không'; 'ks', 'KS', 'Ks' -> 'khách sạn'
def replace_shortened_word1(txt):
  short_word1 = ['K', 'k', 'khong', 'ko', 'kg']
  short_word2 = ['ks', 'KS']
  txt_split = txt.split(' ')
  tmp = ''
  for word in txt_split:
    if word in short_word1:
      tmp += ' ' + 'không'
    elif word in short_word2:
      tmp += ' ' + 'khách sạn'
    else:
      tmp += ' ' + word
  return tmp.strip()

# clean input review
def clean(txt):
  txt = replace_money(txt)
  txt = change_emoijs(txt)
  txt = remove_special_char(txt)
  txt = replace_shortened_word1(txt)
  return " ".join(txt.split())


In [ ]:
# test clean input data
# txt = 'Ấn tượng đầu là quán siêu to siêu thoáng lại còn bonus thêm quả máy hút mùi ở mỗi bàn. Quá hiện đại luôn ! - Vừa đặt mông xuống phát gọi món, chưa đầy 5p là đã ra luôn đồ rồi. Thêm điểm cộng nhân viên vui tính, nhiệt tình lớm 😎- Trong ảnh mình đi 5 người, ăn hết nướng xog lại gọi thêm nồi lẩu thái nữa. Chia ra chưa đến 100k mỗi đứa quá rẻ luôn hihu.- Menu cực kì nhiều món nên rất dễ confuse trong việc chọn 😂 Recommend các bạn gọi sốt Korea ngon cực. Hôm ấy vì ngon quá nên mình chỉ gọi mỗi sốt đó cho tất cả các loại thịt đấy =))))) giá chia ra đầu người quá rẻ luôn rồi.'
# clean(txt)

# 2. Model BERT AD (AP) pre-trained based single sentence classifier model

In [ ]:
# restaurant phase A
target_list = ['FOOD#STYLE&OPTIONS', 'FOOD#QUALITY',
       'AMBIENCE#GENERAL', 'RESTAURANT#GENERAL', 'SERVICE#GENERAL',
       'FOOD#PRICES', 'RESTAURANT#PRICES', 'LOCATION#GENERAL',
       'RESTAURANT#MISCELLANEOUS', 'DRINKS#STYLE&OPTIONS', 'DRINKS#PRICES',
       'DRINKS#QUALITY']

In [ ]:
# # hotel  phase A
# target_list = ['HOTEL#DESIGN&FEATURES', 'HOTEL#GENERAL',
#        'LOCATION#GENERAL', 'SERVICE#GENERAL', 'HOTEL#COMFORT',
#        'HOTEL#CLEANLINESS', 'FACILITIES#GENERAL', 'ROOMS#CLEANLINESS',
#        'ROOM_AMENITIES#COMFORT', 'ROOMS#COMFORT', 'FACILITIES#PRICES',
#        'ROOM_AMENITIES#GENERAL', 'FOOD&DRINKS#STYLE&OPTIONS', 'ROOMS#PRICES',
#        'ROOMS#DESIGN&FEATURES', 'ROOMS#GENERAL', 'HOTEL#QUALITY',
#        'ROOMS#QUALITY', 'FOOD&DRINKS#QUALITY', 'HOTEL#PRICES',
#        'FACILITIES#DESIGN&FEATURES', 'FACILITIES#QUALITY',
#        'ROOM_AMENITIES#QUALITY', 'ROOM_AMENITIES#CLEANLINESS',
#        'ROOM_AMENITIES#DESIGN&FEATURES', 'HOTEL#MISCELLANEOUS',
#        'FOOD&DRINKS#PRICES', 'FACILITIES#COMFORT', 'FOOD&DRINKS#MISCELLANEOUS',
#        'FACILITIES#CLEANLINESS', 'FACILITIES#MISCELLANEOUS',
#        'ROOMS#MISCELLANEOUS', 'ROOM_AMENITIES#MISCELLANEOUS',
#        'ROOM_AMENITIES#PRICES']
# len(target_list)

In [ ]:
# res phase B
# target_list = ['FOOD#STYLE&OPTIONS#neutral', 'FOOD#QUALITY#neutral',
#        'AMBIENCE#GENERAL#neutral', 'RESTAURANT#GENERAL#neutral',
#        'SERVICE#GENERAL#neutral', 'FOOD#PRICES#neutral',
#        'RESTAURANT#PRICES#neutral', 'LOCATION#GENERAL#neutral',
#        'RESTAURANT#MISCELLANEOUS#neutral', 'DRINKS#STYLE&OPTIONS#neutral',
#        'DRINKS#PRICES#neutral', 'DRINKS#QUALITY#neutral',
#        'FOOD#STYLE&OPTIONS#negative', 'FOOD#QUALITY#negative',
#        'AMBIENCE#GENERAL#negative', 'RESTAURANT#GENERAL#negative',
#        'SERVICE#GENERAL#negative', 'FOOD#PRICES#negative',
#        'RESTAURANT#PRICES#negative', 'LOCATION#GENERAL#negative',
#        'RESTAURANT#MISCELLANEOUS#negative', 'DRINKS#STYLE&OPTIONS#negative',
#        'DRINKS#PRICES#negative', 'DRINKS#QUALITY#negative',
#        'FOOD#STYLE&OPTIONS#positive', 'FOOD#QUALITY#positive',
#        'AMBIENCE#GENERAL#positive', 'RESTAURANT#GENERAL#positive',
#        'SERVICE#GENERAL#positive', 'FOOD#PRICES#positive',
#        'RESTAURANT#PRICES#positive', 'LOCATION#GENERAL#positive',
#        'RESTAURANT#MISCELLANEOUS#positive', 'DRINKS#STYLE&OPTIONS#positive',
#        'DRINKS#PRICES#positive', 'DRINKS#QUALITY#positive']

In [ ]:
# #hos phase B
# target_list = ['FACILITIES#CLEANLINESS positive',
#        'FACILITIES#COMFORT positive', 'FACILITIES#DESIGN&FEATURES negative',
#        'FACILITIES#DESIGN&FEATURES positive', 'FACILITIES#GENERAL positive',
#        'FACILITIES#QUALITY negative', 'FACILITIES#QUALITY positive',
#        'FOOD&DRINKS#QUALITY negative', 'FOOD&DRINKS#QUALITY positive',
#        'FOOD&DRINKS#STYLE&OPTIONS negative',
#        'FOOD&DRINKS#STYLE&OPTIONS positive', 'HOTEL#CLEANLINESS positive',
#        'HOTEL#COMFORT negative', 'HOTEL#COMFORT positive',
#        'HOTEL#DESIGN&FEATURES negative', 'HOTEL#DESIGN&FEATURES positive',
#        'HOTEL#GENERAL negative', 'HOTEL#GENERAL neutral',
#        'HOTEL#GENERAL positive', 'HOTEL#MISCELLANEOUS negative',
#        'HOTEL#PRICES negative', 'HOTEL#PRICES positive',
#        'HOTEL#QUALITY negative', 'HOTEL#QUALITY positive',
#        'LOCATION#GENERAL negative', 'LOCATION#GENERAL positive',
#        'ROOMS#CLEANLINESS negative', 'ROOMS#CLEANLINESS positive',
#        'ROOMS#COMFORT negative', 'ROOMS#COMFORT positive',
#        'ROOMS#DESIGN&FEATURES negative', 'ROOMS#DESIGN&FEATURES positive',
#        'ROOMS#GENERAL positive', 'ROOMS#PRICES positive',
#        'ROOMS#QUALITY positive', 'ROOM_AMENITIES#DESIGN&FEATURES negative',
#        'ROOM_AMENITIES#DESIGN&FEATURES positive',
#        'ROOM_AMENITIES#GENERAL positive', 'ROOM_AMENITIES#QUALITY negative',
#        'SERVICE#GENERAL negative', 'SERVICE#GENERAL neutral',
#        'SERVICE#GENERAL positive']
# len(target_list)

In [ ]:
MAX_LEN = 256
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
EPOCHS = 10
LEARNING_RATE = 1e-05
NUM_LABELS = len(target_list)

In [ ]:
from transformers import BertTokenizer, BertModel
from transformers import AutoTokenizer, AutoModel

In [ ]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [ ]:
class BERTClass(nn.Module):
  def __init__(self):
    super(BERTClass, self).__init__()
    # self.bert_model = BertModel.from_pretrained('bert-base-multilingual-cased', return_dict=True)
    self.bert_model = AutoModel.from_pretrained('vinai/phobert-base', return_dict=True)

    self.dropout = nn.Dropout(0.1)
    self.linear = nn.Linear(768, NUM_LABELS) 
  def forward(self, input_ids, attention_mask, token_type_ids):
    output = self.bert_model(input_ids, attention_mask, token_type_ids)
    output_dropout = self.dropout(output.pooler_output)
    output = self.linear(output_dropout)
    return output

In [ ]:
model = BERTClass()
model.load_state_dict(torch.load('/content/drive/MyDrive/Đồ án tốt nghiệp/Save Pre-Train Model/phobertResPhaseA.pt'))
# model.load_state_dict(torch.load('/content/drive/MyDrive/Đồ án tốt nghiệp/Save Pre-Train Model/phobertResPhaseB.pt'))
# model.load_state_dict(torch.load('/content/drive/MyDrive/Đồ án tốt nghiệp/Save Pre-Train Model/phobertHosPhaseA.pt'))
# model.load_state_dict(torch.load('/content/drive/MyDrive/Đồ án tốt nghiệp/Save Pre-Train Model/phobertHosPhaseB.pt'))
model.to(device)

In [ ]:
def infer_AD(review):
  example = clean(review)
  # print(example)
  encodings = tokenizer.encode_plus(
      example,
      None,
      add_special_tokens=True,
      max_length=MAX_LEN,
      padding='max_length',
      return_token_type_ids=True,
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt'
  )
  model.eval()

  with torch.no_grad():
      input_ids = encodings['input_ids'].to(device, dtype=torch.long)
      attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
      token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
      output = model(input_ids, attention_mask, token_type_ids)
      final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
      # print(train.columns[1:].to_list()[int(np.argmax(final_output, axis=1))])
      t = np.array(final_output)
      THRESHOLD = 0.45
      upper, lower = 1, 0

      final_out = np.where(t > THRESHOLD, upper, lower)
      # print(final_out)
      output = [target_list[i] for i in range(len(final_out[0])) if final_out[0][i] == 1]

  # return final_out.tolist()
  return output

In [ ]:
# test
# review = 'Ấn tượng đầu là quán siêu to siêu thoáng lại còn bonus thêm quả máy hút mùi ở mỗi bàn. Quá hiện đại luôn ! - Vừa đặt mông xuống phát gọi món, chưa đầy 5p là đã ra luôn đồ rồi. Thêm điểm cộng nhân viên vui tính, nhiệt tình lớm 😎- Trong ảnh mình đi 5 người, ăn hết nướng xog lại gọi thêm nồi lẩu thái nữa. Chia ra chưa đến 100k mỗi đứa quá rẻ luôn hihu.- Menu cực kì nhiều món nên rất dễ confuse trong việc chọn 😂 Recommend các bạn gọi sốt Korea ngon cực. Hôm ấy vì ngon quá nên mình chỉ gọi mỗi sốt đó cho tất cả các loại thịt đấy =))))) giá chia ra đầu người quá rẻ luôn rồi.'
# aspect_pre= infer_AD(review)

Ấn tượng đầu là quán siêu to siêu thoáng lại còn bonus thêm quả máy hút mùi ở mỗi bàn. Quá hiện đại luôn Vừa đặt mông xuống phát gọi món, chưa đầy 5p là đã ra luôn đồ rồi. Thêm điểm cộng nhân viên vui tính, nhiệt tình lớm thích Trong ảnh mình đi 5 người, ăn hết nướng xog lại gọi thêm nồi lẩu thái nữa. Chia ra chưa đến giá tiền mỗi đứa quá rẻ luôn hihu. Menu cực kì nhiều món nên rất dễ confuse trong việc chọn Recommend các bạn gọi sốt Korea ngon cực. Hôm ấy vì ngon quá nên mình chỉ gọi mỗi sốt đó cho tất cả các loại thịt đấy giá chia ra đầu người quá rẻ luôn rồi.


# 3. Predict polarity by pre-trained BERT POLARITY sentence pair classification modal 

In [ ]:
tokenizer_pair = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [ ]:
target_polarity  = ['positive', 'negative', 'neutral']

In [ ]:
class BERTClassPolarity(nn.Module):
  def __init__(self):
    super(BERTClassPolarity, self).__init__()
    self.bert_model = BertModel.from_pretrained('bert-base-multilingual-cased', return_dict=True)
    # self.bert_model = AutoModel.from_pretrained('vinai/phobert-base', return_dict=True)

    self.dropout = nn.Dropout(0.1)
    self.linear = nn.Linear(768, 3) 
  def forward(self, input_ids, attention_mask, token_type_ids):
    output = self.bert_model(input_ids, attention_mask, token_type_ids)
    output_dropout = self.dropout(output.pooler_output)
    output = self.linear(output_dropout)
    return output

In [ ]:
model_polarity = BERTClassPolarity()
model_polarity.load_state_dict(torch.load('/content/drive/MyDrive/Đồ án tốt nghiệp/Save Pre-Train Model/ResCombine.pt'))
# model_polarity.load_state_dict(torch.load('/content/drive/MyDrive/Đồ án tốt nghiệp/Save Pre-Train Model/HosCombine.pt'))
model_polarity.to(device)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTClassPolarity(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [ ]:
def cleanAspect(x):
  x  = x.replace('[', '')
  x = x.replace("]", '')
  return x.split(',')

def processAspect(x):
  
  x = x.replace('#', ' ')
  x = x.replace('&', ' ')
  return ''.join(x)

In [ ]:
def infer_polarity(review, aspect):
  text_a = review
  text_b = processAspect(aspect)
  # print(text_b)
  encodings = tokenizer_pair.encode_plus(
      text_a,
      text_b,
      add_special_tokens=True,
      max_length=MAX_LEN,
      padding='max_length',
      return_token_type_ids=True,
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt'
  )
  model_polarity.eval()

  with torch.no_grad():
      input_ids = encodings['input_ids'].to(device, dtype=torch.long)
      attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
      token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
      output = model_polarity(input_ids, attention_mask, token_type_ids)
      final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
      # print(train.columns[1:].to_list()[int(np.argmax(final_output, axis=1))])
      t = np.array(final_output)
      THRESHOLD = 0.5
      upper, lower = 1, 0

      final_out = np.where(t > THRESHOLD, upper, lower)
      output = [target_polarity[i] for i in range(len(final_out[0])) if final_out[0][i] == 1]

  return final_out.tolist()

In [ ]:
# test polarity
# result_test = []
# for i in  range(len(df_test['content'].values)):
#   infer_test = infer_polarity(str(df_test['content'][i]), str(df_test['aspect'][i]))
#   list_aspect = [target_polarity[i] for i in range(len(infer_test[0])) if infer_test[0][i] == 1]
#   result_test.append(list_aspect)

# 4. Main

In [ ]:
def main_AD(review):
  example = clean(review)
  aspect_pre = infer_AD(review)
  return ','.join(aspect_pre)
  
def main_Polarity(review):
  example = clean(review)
  aspect_pre =  main_AD(example).split(',')
  result  = [[review, aspect_pre]]
  df_test = pd.DataFrame(result, columns=['content', 'aspect'])
  df_test = df_test.explode('aspect')
  df_test.reset_index(inplace=True)
  polarity_pre = []
  for i in  range(len(df_test['content'].values)):
    infer_test = infer_polarity(str(df_test['content'][i]), str(df_test['aspect'][i]))
    list_polarity = [target_polarity[i] for i in range(len(infer_test[0])) if infer_test[0][i] == 1]
    if len(list_polarity) > 1:
      polarity = list_polarity[0]
    else:
      polarity = 'positive'
    polarity_pre.append(polarity)
  df_test['polarity'] = polarity_pre
  df_test['combine'] = df_test['aspect'] + '#' + df_test['polarity']
  return df_test

In [ ]:
review = 'Quán j đâu mà xứt xắc z troiii đồ ăn ngon xỉu lùn á:33'
# review = 'Ăn khá nhiều nhà hàng nhật rồi.nhưng lại ưng nhà hàng này nhất. Đồ ăn đa dạng, ngon miệng, nhân viên nhiệt tình,đồ ăn phục vụ nhanh chóng.sẽ giới thiệu bạn bè quay trở lạ'

In [ ]:
main_AD(review)

'FOOD#QUALITY'

In [ ]:
main_Polarity(review)

,index,content,aspect,polarity,combine
0,0,Quán j đâu mà xứt xắc z troiii đồ ăn ngon xỉu ...,FOOD#QUALITY,positive,FOOD#QUALITY#positive
